# Query de usuario, extraer transcripciones relevantes y respuesta de LLM.

In [ ]:
import openai

def retrieve_and_generate_response(user_query):
    # Convert user query into an embedding
    query_embedding = openai.Embedding.create(input=user_query, model="text-embedding-ada-002")["data"][0]["embedding"]

    # Search Pinecone for similar transcriptions
    search_results = index.query(vector=query_embedding, top_k=3, include_metadata=True)

    # Retrieve the most relevant transcriptions
    retrieved_texts = [match["metadata"]["text"] for match in search_results["matches"]]

    # Combine retrieved texts for context
    context = "\n".join(retrieved_texts)

    # Pass retrieved context to DeepSeek LLM
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Replace with "deepseek" if using DeepSeek LLM
        messages=[
            {"role": "system", "content": "You are an AI assistant that helps call center agents improve their interactions."},
            {"role": "user", "content": f"Context:\n{context}\n\nUser Question: {user_query}"}
        ]
    )

    return response["choices"][0]["message"]["content"]

# Example Query
user_input = "What was the most common customer complaint?"
answer = retrieve_and_generate_response(user_input)
print("Chatbot Response:", answer)
